In [2]:
import time
import numpy
import matplotlib.pyplot as plt
import pandas
import math
import os
from keras.models import Sequential
from keras.callbacks import TensorBoard
from keras.layers import Dense ,Activation
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras import metrics
from IPython.core.debugger import Tracer
%matplotlib inline

In [3]:
# fix random seed for reproducibility
numpy.random.seed(10)
cwd = os.getcwd()
os.chdir("C:/Users/mukul.virmani/")
path = './Desktop/CPI_US_Medical.csv'
scaler = MinMaxScaler()
#get Transformed data after 
def getTansformedData(path):
    frame = pandas.read_csv(path,engine='python',skiprows=0)
    scaler = MinMaxScaler(feature_range=(0, 1))
    frame['value'] = scaler.fit_transform(frame.value.values.reshape(-1,1))
    return frame

In [4]:
data=getTansformedData(path)
def num_missing(x):
  return sum(x.isnull())
data.apply(num_missing, axis=0)
#get_null=num_missing(data)

series_id    0
year         0
period       0
value        0
Month        0
Date         0
dtype: int64

In [5]:
data

,series_id,year,period,value,Month,Date
0,CUSR0000SAM,1947,M01,0.000000,1,1/1/1947
1,CUSR0000SAM,1947,M02,0.000216,2,2/1/1947
2,CUSR0000SAM,1947,M03,0.000216,3,3/1/1947
3,CUSR0000SAM,1947,M04,0.000432,4,4/1/1947
4,CUSR0000SAM,1947,M05,0.000648,5,5/1/1947
5,CUSR0000SAM,1947,M06,0.000648,6,6/1/1947
6,CUSR0000SAM,1947,M07,0.000648,7,7/1/1947
7,CUSR0000SAM,1947,M08,0.000864,8,8/1/1947
8,CUSR0000SAM,1947,M09,0.001079,9,9/1/1947
9,CUSR0000SAM,1947,M10,0.001295,10,10/1/1947


In [11]:
splitfactor,prev=.80,4
# convert an array of values into a dataset matrix and create sliding window approach.
def Get_rolling_window_data(d1,prev):
    x,y,val=[],[],len(d1)-prev
    for z in range(val-1):
        x.append(d1.values[z:(z + prev),3 ]), y.append(d1.values[z + prev, 3])
    return numpy.array(x), numpy.array(y)

In [12]:
def data_reshape(d1, splitfactor,prev):
    # split into train and test sets
    data_train,data_test = int(len(d1) *splitfactor ),int(len(d1) *(1-splitfactor))
    train, test = d1.iloc[0:data_train,:], d1.iloc[data_train:len(d1),:]

    # reshape into X=t and Y=t+1
    train_X, train_Y = Get_rolling_window_data(train, prev)
    testX, testY=Get_rolling_window_data(test, prev)
   
    # reshape input to be [samples, time steps, features]
    train_X = numpy.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))
    testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))
    return train_X,testX, train_Y, testY

train_X,testX, train_Y, testY=data_reshape(data,splitfactor,prev)
print (len(train_X),len(testX))
print(train_X,testX)

672 165
[[[0.0]
  [0.00021589738828929964]
  [0.00021589738828929964]
  [0.00043179477657859233]]

 [[0.00021589738828929964]
  [0.00021589738828929964]
  [0.00043179477657859233]
  [0.000647692164867885]]

 [[0.00021589738828929964]
  [0.00043179477657859233]
  [0.000647692164867885]
  [0.000647692164867885]]

 ..., 
 [[0.5971721760081867]
  [0.6001947394442367]
  [0.6021378159388405]
  [0.6034332002685762]]

 [[0.6001947394442367]
  [0.6021378159388405]
  [0.6034332002685762]
  [0.6040808924334441]]

 [[0.6021378159388405]
  [0.6034332002685762]
  [0.6040808924334441]
  [0.6049444819866012]]] [[[0.6107737114704123]
  [0.613580377518173]
  [0.6155234540127766]
  [0.6187619148371161]]

 [[0.613580377518173]
  [0.6155234540127766]
  [0.6187619148371161]
  [0.6200572991668518]]

 [[0.6155234540127766]
  [0.6187619148371161]
  [0.6200572991668518]
  [0.6222162730497447]]

 [[0.6187619148371161]
  [0.6200572991668518]
  [0.6222162730497447]
  [0.6256706312623734]]

 [[0.6200572991668518]
 

In [13]:
def build_model_FINAL(train_X, train_Y,prev):
    model = Sequential()
    batch_size=1
    model.add(SimpleRNN(4, batch_input_shape=(batch_size, prev, 1), stateful=True, return_sequences=True))
    model.add(LSTM(4, batch_input_shape=(batch_size,prev, 1), stateful=True, return_sequences=True))
    model.add(LSTM(6,batch_input_shape=(batch_size,prev,1),stateful=True))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(1):
        model.fit(train_X, train_Y, epochs=20, batch_size=batch_size, verbose=2,shuffle=False)
        model.reset_states()
    return model

model=build_model_FINAL(train_X, train_Y,prev)

Epoch 1/20
6s - loss: 4.6788e-05
Epoch 2/20
4s - loss: 0.0011
Epoch 3/20
4s - loss: 0.0013
Epoch 4/20
4s - loss: 6.0478e-04
Epoch 5/20
4s - loss: 1.5957e-04
Epoch 6/20
4s - loss: 1.2154e-04
Epoch 7/20
3s - loss: 8.6188e-05
Epoch 8/20
3s - loss: 5.1209e-05
Epoch 9/20
3s - loss: 1.6221e-04
Epoch 10/20
3s - loss: 9.6188e-05
Epoch 11/20
3s - loss: 7.9686e-05
Epoch 12/20
3s - loss: 7.2339e-05
Epoch 13/20
3s - loss: 3.1394e-05
Epoch 14/20
3s - loss: 3.0618e-05
Epoch 15/20
4s - loss: 3.4925e-05
Epoch 16/20
3s - loss: 3.6282e-05
Epoch 17/20
4s - loss: 1.4610e-05
Epoch 18/20
3s - loss: 1.6627e-05
Epoch 19/20
4s - loss: 4.9784e-05
Epoch 20/20
4s - loss: 4.8547e-05


In [14]:
batch_size=1
def Get_Inverse_transform(x):
    fit_1=scaler.fit(x)
    return scaler.inverse_transform(x)
# make predictions
predtr = Get_Inverse_transform(model.predict(train_X,batch_size))
predtr[predtr < 0] = 0
train_Y =  Get_Inverse_transform([train_Y])
predte,testY = Get_Inverse_transform(model.predict(testX, batch_size)),Get_Inverse_transform([testY])
predte[predte < 0] = 0

In [15]:
def Find_RMSE_MAE(x,x1,i):
    if i==1:
        return math.sqrt(mean_squared_error(x, x1))
    else:
        return mean_absolute_error(x, x1)
# calculate root mean squared error
trerr= Find_RMSE_MAE(train_Y[0],predtr[:,0],1)
teerr =Find_RMSE_MAE(testY[0], predte[:,0],1)
print('Train Score: %.2f RMSE' %(trerr))
print ('Test Score: %.2f RMSE' % (teerr))
# calculate mean absloute error
meanerrtr = Find_RMSE_MAE(train_Y[0],predtr[:,0],0)
meanerrte = Find_RMSE_MAE(testY[0], predte[:,0],0)
print ('Train Score: %.2f MAE' %(meanerrtr))
print ('Test Score: %.2f MAE' % (meanerrte))

Train Score: 0.32 RMSE
Test Score: 1.02 RMSE
Train Score: 0.21 MAE
Test Score: 1.00 MAE
